<a href="https://colab.research.google.com/github/sssharaf/ml-nlp/blob/master/start_servers_play_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/gdrive')
!ls -ltr /gdrive/'My Drive'/ML/data/start-servers-play
!pip install pytorch_transformers
!ln -s  /gdrive/'My Drive'/ML/data/start-servers-play data
!ls -ltr data/*

In [0]:
import numpy as np
import pandas as pd
from pandas import DataFrame
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader,Dataset
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
import pytorch_transformers as pt
from pytorch_transformers import BertTokenizer, BertConfig,BertForMaskedLM,BertModel,DistilBertTokenizer, DistilBertModel,DistilBertForSequenceClassification 
import os
import typing
from typing import Dict,List,Sequence,Set
from types import SimpleNamespace as SN
import numpy as np
import pickle
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.utils.class_weight import compute_class_weight,compute_sample_weight
T_BertTokenizer = typing.NewType("BertTokenizer",BertTokenizer)
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu') 

In [0]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [0]:
trn_df = pd.read_csv('data/train.csv',dtype={'action':'category','component':'category'},)
val_df = pd.read_csv('data/val-new.csv',dtype={'action':'category','component':'category'})
#trn_df.loc[trn_df.action=='noaction']
trn_df

In [0]:
action_le = LabelEncoder()
action_le.fit(trn_df.action)
component_le = LabelEncoder()
component_le.fit(trn_df.component)
print(action_le.classes_)
print(component_le.classes_)

with open(f'data/action_le.dat','wb') as f:
  pickle.dump(action_le,f)

with open(f'data/component_le.dat','wb') as f:
  pickle.dump(component_le,f)


In [0]:

def encode_X(comment:str,max_len):
  X = f"[CLS] {comment} [SEP]"
  encoded = torch.tensor(tokenizer.encode(X),dtype=torch.long)
  X = torch.zeros(max_len,dtype=torch.long)
  X[:len(encoded)] = encoded
  X[len(encoded)+1:] = torch.tensor(tokenizer.pad_token_id,dtype=torch.long)  
  X_attn_mask = X!=tokenizer.pad_token_id
  X_attn_mask = X_attn_mask.int()
  return X,X_attn_mask

class MyDataset(Dataset):

    def __init__(self,df:DataFrame,max_len = 16):
        self.df = df
        self.max_len=max_len
        self.action = self.df.action.cat.codes
        self.component = self.df.component.cat.codes

    def __getitem__(self,index):
        X = self.df.iloc[index]['comment_text']
        X,X_attn_mask = encode_X(X,self.max_len)
        Y1 = self.df.iloc[index]['action']
        Y1 = action_le.transform([Y1])
        #Y1 = a_ohe.transform([[Y1]])
        Y1 = torch.tensor(Y1,dtype=torch.long)
        Y2 = self.df.iloc[index]['component']
        Y2 = component_le.transform([Y2])
        #Y2 = c_ohe.transform([[Y2]])
        Y2 = torch.tensor(Y2, dtype=torch.long)
        return (X,X_attn_mask),(Y1.squeeze(),Y2.squeeze())

    def __len__(self):
        return len(self.df)

    def components(self):
        return self.component

In [0]:

trn_ds = MyDataset(trn_df,max_len=25)
val_ds = MyDataset(val_df,max_len=25)

trn_dl = DataLoader(dataset=trn_ds,batch_size=16,pin_memory=True,shuffle=True)
val_dl = DataLoader(dataset=val_ds,batch_size=16,pin_memory=True,shuffle=True)


In [0]:
# Model
class MyModel3(nn.Module):
  def __init__(self, freeze_bert = True):
    super().__init__()
    self.model_version = 3

    self.bert_lyr = BertModel.from_pretrained('bert-base-uncased',output_hidden_states=True)
    
    self.a_attn = nn.Linear(768,1,bias=False)
    
    self.c_attn = nn.Linear(768,1,bias=False)

    self.attn_dropout = nn.Dropout(0.1,inplace=False)
    
    self.a_switch = nn.Sequential(
        nn.Dropout(0.1),
        nn.Linear(768*2,64),
        nn.ELU(),
        nn.LayerNorm(64),
        nn.Dropout(0.1),
        nn.Linear(64,1),
        nn.Sigmoid(),
        )
    
    self.c_switch = nn.Sequential(
        nn.Dropout(0.1),
        nn.Linear(768*2,64),
        nn.ELU(),
        nn.LayerNorm(64),
        nn.Dropout(0.1),
        nn.Linear(64,1),
        nn.Sigmoid(),
    )

    self.action_cls_lyr = nn.Sequential(
        nn.Dropout(0.1),
        nn.Linear(768,64,bias=False),
        nn.ELU(),
        nn.LayerNorm(64),
        nn.Dropout(0.1),
        nn.Linear(64,len(action_le.classes_),bias=False),   
    )

    self.component_cls_lyr = nn.Sequential(
                nn.Dropout(0.1),
                nn.Linear(768,64,bias=False),
                nn.ELU(),
                nn.LayerNorm(64),
                nn.Dropout(0.1),
                nn.Linear(64,len(component_le.classes_),bias=False),
    )
    
    #Freeze bert layers
    if freeze_bert:
       for p in self.bert_lyr.parameters():#self.bert_lyr.parameters():
              p.requires_grad = False
    #nn.init.xavier_uniform_(self.action_cls_lyr)
    #nn.init.xavier_uniform_(self.component_cls_lyr)

  def forward(self, seq, attn_masks,output_attn=False,output_hs=False,output_switch=False):
    attn_mask_cls = (1 - attn_masks)*-10000
    attn_mask_cls.unsqueeze_(dim=-1)

    seq_emb,ctx,hs = self.bert_lyr(seq,attention_mask =attn_masks)
   
    a = self.a_attn(seq_emb)
    a = a + attn_mask_cls
    a =  a_output = a.softmax(dim=1)
    a = self.attn_dropout(a)
    a = torch.mul(seq_emb,a)
    a = a.mean(dim=1)

    c = self.c_attn(seq_emb)
    c = c + attn_mask_cls
    c =  c_output = c.softmax(dim=1)
    c = self.attn_dropout(c)
    c = torch.mul(seq_emb,c)
    c = c.mean(dim=1)

    a_switch = self.a_switch(torch.cat([ctx.detach(),a],dim=-1))    
    a = (1 - a_switch)*a + a_switch*ctx

    c_switch = self.c_switch(torch.cat([ctx.detach(),c],dim=-1))
    c = (1 - c_switch)*c + c_switch*ctx

    outputs = [self.action_cls_lyr(a),self.component_cls_lyr(c)]
    if (output_attn):
      outputs += [a_output,c_output]
    if output_hs:
      outputs +=[hs]
    if output_switch:
      outputs +=[a_switch,c_switch]
    return outputs

############################### Model 2  ############################################
# Model
class MyModel2(nn.Module):
  def __init__(self, freeze_bert = True):
    super().__init__()
    self.model_version = 2-1
    #self.static_bert_lyr = BertModel.from_pretrained('bert-base-uncased',output_hidden_states=False)
    self.bert_lyr = BertModel.from_pretrained('bert-base-uncased',output_hidden_states=True)
    
    self.a_attn = nn.Linear(768,1)
    
    self.c_attn = nn.Linear(768,1)
    
    self.action_cls_lyr = nn.Sequential(
        nn.Dropout(0.1),
        nn.Linear(768,64,bias=True),
        nn.LayerNorm(64),
        nn.ELU(),
        nn.Linear(64,len(action_le.classes_),bias=True),
        nn.LayerNorm(len(action_le.classes_)),
        #nn.Linear(768,len(action_le.classes_),bias=False),      
    )

    self.component_cls_lyr = nn.Sequential(
                nn.Dropout(0.1),
                nn.Linear(768,64,bias=True),
                nn.LayerNorm(64),
                nn.ELU(),
                nn.Linear(64,len(component_le.classes_),bias=True),
                nn.LayerNorm(len(component_le.classes_)),
                #nn.Linear(768,len(component_le.classes_),bias=False),
    )
    
    # for p in self.static_bert_lyr.parameters():
    #   p.requires_grad = False

    #Freeze bert layers
    if freeze_bert:
        for lyr in self.bert_lyr.encoder.layer[:-2]:
          for p in lyr.parameters():#self.bert_lyr.parameters():
              p.requires_grad = False
    #nn.init.xavier_uniform_(self.action_cls_lyr)
    #nn.init.xavier_uniform_(self.component_cls_lyr)

  def forward(self, seq, attn_masks,output_attn=False,output_hs=False):
    attn_mask_cls = (1 - attn_masks)*-10000
    attn_mask_cls.unsqueeze_(dim=-1)

    #static_emb,static_ctx = self.static_bert_lyr(seq,attention_mask =attn_masks)
    seq_emb,ctx,hs = self.bert_lyr(seq,attention_mask =attn_masks)
    #seq_emb +=static_emb
    a = self.a_attn(seq_emb)
    a = a + attn_mask_cls
    a = a_output = a.softmax(dim=1)
    a = torch.mul(seq_emb,a)
    a = a.mean(dim=1)

    c = self.c_attn(seq_emb)
    c = c + attn_mask_cls
    c = c_output =  c.softmax(dim=1)
    c = torch.mul(seq_emb,c)
    c = c.mean(dim=1)

    outputs = [self.action_cls_lyr(a),self.component_cls_lyr(c)]
    if (output_attn):
      outputs += [a_output,c_output]
    if output_hs:
      outputs +=[hs]
    return outputs
#####################################################################################
model = MyModel3(freeze_bert=False)
model = model.to(DEVICE)

In [0]:
from pytorch_transformers import BertForSequenceClassification
############################### Model 1  ############################################
# Model
class MyModel4(nn.Module):
  def __init__(self, freeze_bert = True):
    super().__init__()
    self.model_version = 4
    
    self.bert_lyr = BertModel.from_pretrained('bert-base-uncased',output_hidden_states=True,output_attentions=True)

    #self.comp_bert_lyr = BertModel.from_pretrained('bert-base-uncased',output_hidden_states=True,output_attentions=True)
    
    self.config = self.bert_lyr.config;

    self.action_cls_lyr = nn.Sequential(
        nn.Dropout(0.3),
        nn.Linear(768,4),
    )
    self.comp_cls_lyr = nn.Sequential(
            nn.Dropout(0.3),
            nn.Linear(768,5),
        )    

    #Freeze bert layers
    if freeze_bert:
        for p in self.bert_lyr.parameters():
          p.requires_grad = False
        for p in self.comp_bert_lyr.parameters():
          p.requires_grad = False

  def forward(self, seq, attn_masks,output_attn=False,output_hs=False):
    
    seq_emb,pooled,hs,attn = self.bert_lyr(seq,attention_mask =attn_masks)
    
    #c_seq_emb,c_pooled,c_hs,c_attn = self.comp_bert_lyr(seq,attention_mask =attn_masks)
    
    outputs=[]
    outputs += [
                self.action_cls_lyr(pooled),
                self.comp_cls_lyr(pooled),
                ]
    return outputs

try:
  del model
  torch.cuda.reset_max_memory_allocated()
  torch.cuda.reset_max_memory_cached()
except NameError:
  pass

model = MyModel1(freeze_bert=False)
model = model.to(DEVICE)


In [0]:
from pytorch_transformers import BertForSequenceClassification
############################### Model 1  ############################################
# Model
class MyModel5(nn.Module):
  def __init__(self, freeze_bert = True):
    super().__init__()
    self.model_version = 5
    
    self.bert_lyr = BertModel.from_pretrained('bert-base-uncased',output_hidden_states=True,output_attentions=True)

    self.comp_bert_lyr = BertModel.from_pretrained('bert-base-uncased',output_hidden_states=True,output_attentions=True)
    
    self.config = self.bert_lyr.config;

    self.action_cls_lyr = nn.Sequential(
        nn.Dropout(0.3),
        nn.Linear(768,4),
    )
    self.comp_cls_lyr = nn.Sequential(
            nn.Dropout(0.3),
            nn.Linear(768,5),
        )    

    #Freeze bert layers
    if freeze_bert:
        for p in self.bert_lyr.parameters():
          p.requires_grad = False
        for p in self.comp_bert_lyr.parameters():
          p.requires_grad = False

  def forward(self, seq, attn_masks,output_attn=False,output_hs=False):
    
    seq_emb,pooled,hs,attn = self.bert_lyr(seq,attention_mask =attn_masks)
    
    c_seq_emb,c_pooled,c_hs,c_attn = self.comp_bert_lyr(seq,attention_mask =attn_masks)
    
    outputs=[]
    outputs += [
                self.action_cls_lyr(pooled),
                self.comp_cls_lyr(c_pooled),
                ]
    return outputs

try:
  del model
  torch.cuda.reset_max_memory_allocated()
  torch.cuda.reset_max_memory_cached()
except NameError:
  pass

model = MyModel5(freeze_bert=False)
model = model.to(DEVICE)


In [0]:
from pytorch_transformers import BertForSequenceClassification
############################### Model 1  ############################################
# Model
class MyModel7(nn.Module):
  def __init__(self, freeze_bert = True):
    super().__init__()
    self.model_version = '7_1'
    
    self.bert_lyr = BertModel.from_pretrained('bert-base-uncased',output_hidden_states=True,output_attentions=True)

    #self.comp_bert_lyr = BertModel.from_pretrained('bert-base-uncased',output_hidden_states=True,output_attentions=True)
    
    self.config = self.bert_lyr.config;

    self.a_attn = nn.Linear(768,1)
    
    self.c_attn = nn.Linear(768,1)

    self.action_cls_lyr = nn.Sequential(
        nn.Dropout(0.3),
        nn.Linear(768,4),
    )
    self.comp_cls_lyr = nn.Sequential(
            nn.Dropout(0.3),
            nn.Linear(768,5),
        )    

    #Freeze bert layers
    if freeze_bert:
        self.freeze_bert()
    else:
        self.freeze_bert()
        self.unfreeze_bert()
        
  def freeze_bert(self):
      for p in self.bert_lyr.parameters():
        p.requires_grad = False
      # for p in self.comp_bert_lyr.parameters():
      #   p.requires_grad = False

  def unfreeze_bert(self,from_lyr=6):

    for lyr in self.bert_lyr.encoder.layer[-6:]:
      for p in lyr.parameters():
        p.requires_grad = True
    # for lyr in self.comp_bert_lyr.encoder.layer[-6:]:
    #   for p in lyr.parameters():
    #     p.requires_grad = True


  def forward(self, seq, attn_masks,output_attn=False,output_hs=False):
    attn_mask_cls = (1 - attn_masks)*-10000
    attn_mask_cls.unsqueeze_(dim=-1)
    seq_emb,pooled,hs,attn = self.bert_lyr(seq,attention_mask =attn_masks)
    
    #c_seq_emb,c_pooled,c_hs,c_attn = self.comp_bert_lyr(seq,attention_mask =attn_masks)
    a,a_output = self.attention(seq_emb,self.a_attn,attn_mask_cls)

    c,c_output = self.attention(seq_emb,self.c_attn,attn_mask_cls)

    a_pooled  = a
    c_pooled  = c
    outputs=[]
    outputs += [
                self.action_cls_lyr(a_pooled),
                self.comp_cls_lyr(c_pooled),
                ]
    if (output_attn):
      outputs += [a_output,c_output]
    return outputs

  def attention(self,seq_emb,attn_lyr,attn_mask_cls):
    a = attn_lyr(seq_emb)
    a = a + attn_mask_cls
    a = a_output = a.softmax(dim=1)
    #a_output = a.clone()
    a = torch.mul(seq_emb,a)
    a = a.mean(dim=1)
    return a,a_output

try:
  del model
  torch.cuda.reset_max_memory_allocated()
  torch.cuda.reset_max_memory_cached()
except NameError:
  pass

model = MyModel7(freeze_bert=False)
model = model.to(DEVICE)


In [0]:
print(f'data/model-{model.model_version}.dat')

In [0]:
(X, X_attn),(_,_) = next(iter(trn_dl))
X, X_attn = X.to(DEVICE),X_attn.to(DEVICE)
a = model(X,X_attn,output_attn=True,output_hs=True)


In [0]:
# Load model
model = MyModel7(freeze_bert=False)
#model.load_state_dict(torch.load(f'data/model-{model.model_version}.dat'))
model.to(DEVICE)

g_wd = 1
g_lr = 1e-4
optimizer = torch.optim.AdamW([
                               {'params':model.bert_lyr.parameters(),'lr':5e-6,'weight_decay':0.1},
                              #  {'params':model.comp_bert_lyr.parameters(),'lr':5e-6,'weight_decay':0.1},
                                {'params':model.a_attn.parameters()},
                                {'params':model.c_attn.parameters()},
                               {'params':model.action_cls_lyr.parameters()},
                               {'params':model.comp_cls_lyr.parameters(),},
                               ],lr=g_lr,weight_decay=g_wd )
scheduler = CosineAnnealingWarmRestarts(optimizer,T_0=20,T_mult=1)

In [0]:
# Train model
# torch.cuda.reset_max_memory_allocated()
# torch.cuda.reset_max_memory_cached()
print(f'Device type is {DEVICE.type}')
model.to(DEVICE)
model.train()
le_trnf = action_le.transform(trn_df.action)
u,c = np.unique(le_trnf,return_counts=True)
ac_class_weight=compute_class_weight('balanced',classes=u,y=le_trnf)
ac_class_weight = torch.tensor(ac_class_weight,dtype=torch.float,device=DEVICE)
le_trnf = component_le.transform(trn_df.component)
u,c = np.unique(le_trnf,return_counts=True)
com_class_weight=compute_class_weight('balanced',classes=u,y=le_trnf)
com_class_weight = torch.tensor(com_class_weight,dtype=torch.float,device=DEVICE)

print(f'ac_class_weight={ac_class_weight}')
print(f'com_class_weight={com_class_weight}')


action_criterion = nn.modules.loss.CrossEntropyLoss(weight=ac_class_weight,reduction='mean')
component_criterion = nn.modules.loss.CrossEntropyLoss(weight=com_class_weight,reduction='mean')

n_epochs = 2000

def evaluate_model(epoch:int,model:nn.Module,dl:DataLoader,optimizer):
  t_loss=0.0
  t_a_loss=0.0
  t_c_loss=0.0
  iters = len(dl)
  for i,((X, attn_mask),(Y1,Y2)) in enumerate(dl):
    X , attn_mask,Y1,Y2 = X.to(DEVICE),attn_mask.to(DEVICE),Y1.to(DEVICE),Y2.to(DEVICE)
    p_a,p_c = model(X,attn_mask)
    action_loss = action_criterion(p_a.view(-1,4),Y1)
    component_loss = component_criterion(p_c.view(-1,5),Y2)
    loss =   action_loss + component_loss
    if optimizer is not None:
      scheduler.step(epoch+i/iters)
      optimizer.zero_grad()
      torch.nn.utils.clip_grad_norm_(model.parameters(),5)
      loss.backward()
      optimizer.step()
    t_loss += loss.item()
    t_a_loss += action_loss.item()
    t_c_loss += component_loss.item()
    
  return t_loss,t_a_loss,t_c_loss

for epoch in range(n_epochs):
  t_loss=0
  a_loss=0
  c_loss=0
  t_loss,a_loss,c_loss = evaluate_model(epoch,model,trn_dl,optimizer)
  if epoch%10 == 0:
    torch.save(model.state_dict(),f'data/model-{model.model_version}.dat')
  v_loss = 0
  with torch.no_grad():
    v_loss = evaluate_model(epoch,model,val_dl,None)
    v_loss = [f'{v:0.4}' for v in v_loss]
  print(f'Epoch:{epoch} Trn loss={t_loss:0.4}, Actn loss:{a_loss:0.4} ,Comp loss:{c_loss:0.4},  Validation loss:{v_loss}')


In [0]:
def print_tensor(t:torch.tensor):
  t = t.squeeze().detach().cpu().numpy()
  t = [f'{item:0.3}' for item in t]
  print(t)

model.eval()
with torch.no_grad():
  X,X_attn_mask = encode_X('',max_len=20)
  
  X_tokenized = X
  X,X_attn_mask = X.to(DEVICE), X_attn_mask.to(DEVICE)
  X.unsqueeze_(0)
  X_attn_mask.unsqueeze_(0)
  #action,component,a_attn,c_attn,a_switch,c_switch = model(X,X_attn_mask,output_attn=True,output_switch=True)
  action,component = model(X,X_attn_mask,output_attn=False)
  action = action.softmax(dim=1)
  component = component.softmax(dim=1)
  action,component = [a.squeeze().detach().cpu().numpy() for a in [action,component]]

  print(action_le.classes_)  
  print(action)
  #print(a_attn.squeeze())
  action = np.argmax(action)
  print(f'Selected Action: {action_le.inverse_transform([action])}')

  print(component_le.classes_)
  print(component)
  component = np.argmax(component)
  print(component)
  print(f'Selected component: {component_le.inverse_transform([component])}')

  print(tokenizer.convert_ids_to_tokens(X_tokenized.numpy()))
  #print_tensor(a_attn)
  #print_tensor(c_attn)
  #print(a_attn.squeeze().detach().cpu().numpy())
  #print(c_attn.squeeze().detach().cpu().numpy())


In [0]:
# [p for p in iter(model.action_cls_lyr.named_parameters())]